In [ ]:
from keras.models import load_model
from metrics import Pearson
from utils import prepare_data
import tensorflow_probability as tfp
from metrics import pearson, Pearson
import tensorflow as tf
import numpy as np
from dataloaders import GeneDataLoader
from utils import prepare_data, set_variables
from models import CNN
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Nadam
from keras.losses import KLDivergence, CategoricalCrossentropy
from plotting import plot_line_graph, multiplot_pearson
from utils import prepare_data, set_variables
from metrics import pearson, Pearson
from models import CNN
from plotting import box_plot
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:


train_data, valid_data, test_data = prepare_data(colab=True)



# LOADING model

In [ ]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()
model_name = "CNN_Baseline_4Conv_Struct_ext_struct_m6a" # "steve_CNN_Att10" #'CNN_architecture7'
model_architecture_path, model_output_path, params_dataLoader_train, params_dataLoader_valid, params_model, params_train = set_variables(model_name, max_seq_len)


In [ ]:

model = CNN(input_size=(max_seq_len, 6),
            params_model=params_model,
            optimizer=Nadam(),
            #loss=correlation_coefficient_loss, #KLDivergence(),
            metrics=[pearson, Pearson(name="class_pearson", return_dict=True),  Pearson(name="class_nodict_pearson", return_dict=False), KLDivergence(name="kullback_leibler_divergence"), "accuracy"],
            )
model.print_model()

In [ ]:
path = "INPUTPATH"
model.load_weights(path)

model = load_model('/Users/jonasflor/Downloads/Baseline_m6A.h5',
                   custom_objects={'Pearson': Pearson})

In [ ]:
dataloader = GeneDataLoader(test_data, padding_length=train_data['seq'].apply(lambda x: len(x)).max(), shuffle=False, struct=True)

prediction = []
y_test = []

for i, j in dataloader:
    i_0 = np.delete(i[0], 5, axis=2)
    i_1 = i[1]
    prediction.append(model.predict([i_0, i_1]))
    y_test.append(j)

prediction = np.concatenate(prediction)
y_test = np.concatenate(y_test)

In [ ]:
def pearson_dict(y_true, y_pred):
    y_true_std = tfp.stats.stddev(y_true, sample_axis=0, keepdims=True)
    y_pred_std = tfp.stats.stddev(y_pred, sample_axis=0, keepdims=True)
    
    y_true /= (y_true_std + 1e-3)
    y_pred /= (y_pred_std + 1e-3)
    
    result = tfp.stats.covariance(x=y_true,
                                y=y_pred,
                                event_axis=None,
                                sample_axis=0,
                                keepdims=False)
    res_dict = {}
    res_dict['ERM'] = result[0]
    res_dict['KDEL'] = result[1]
    res_dict['LMA'] = result[2]
    res_dict['MITO'] = result[3]
    res_dict['NES'] = result[4]
    res_dict['NIK'] = result[5]
    res_dict['NLS'] = result[6]
    res_dict['NUCP'] = result[7]
    res_dict['OMM'] = result[8]
    return res_dict

In [ ]:


pearson_dict(y_test, prediction)

